In [47]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [48]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [49]:
path = 'data_prolific'
subject_files = os.listdir(path)
for i in range(0, len(subject_files)):
    thisSubject = open(path + "/" + subject_files[i]).read()
    if thisSubject.find('5c95970cd676900016e1a940') > (-1):
        print(subject_files[i])

research-study-prolific-168.csv
research-study-prolific-178.csv
research-study-prolific-181.csv
research-study-prolific-183.csv


## Read CSV from String

In [50]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

data_raw = compileData("data_prolific")

# Custom modifications
data_raw['run_id'].unique()

array([1, 103, 105, 106, 108, 11, 117, 12, 121, 124, 125, 126, 128, 13,
       130, 131, 14, 141, 143, 144, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 156, 157, 160, 161, 162, 163, 164, 165, 166, 168, 171,
       172, 173, 176, 177, 178, 179, 180, 181, 183, 185, 186, 188, 189,
       19, 190, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206,
       207, 208, 209, 210, 211, 212, 213, 24, 25, 28, 3, 30, 32, 36, 37,
       38, 4, 41, 42, 43, 45, 47, 48, 49, 5, 50, 51, 54, 56, 57, 58, 59,
       6, 61, 63, 66, 67, 7, 70, 72, 74, 75, 80, 81, 83, 85, 88, 9, 90,
       91, 92, 93, 94, 96, 97, 98, 99], dtype=object)

In [51]:
#data_yang = compileData("data_yang2020WG")
# Custom modifications
#data_yang.loc[0:515, 'run_id'] = 0
#data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
#data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
#data_yang['run_id'] = data_yang['run_id'] + 1000
#print(data_yang['run_id'].unique())
#data_raw = data_raw.append(data_yang.reset_index(drop=True))

In [52]:
#data_milieu = compileData("data_milieu")
#data_milieu['run_id'] = data_milieu['run_id'] + 2000
#print(data_milieu['run_id'].unique())
#data_raw = data_raw.append(data_milieu.reset_index(drop=True))

### Exclude empty studies

In [53]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [54]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

[1 103 106 108 11 12 124 125 126 128 13 130 131 14 141 144 146 147 148 149
 150 151 152 153 154 156 157 160 161 162 163 164 165 166 168 171 172 173
 176 177 178 179 180 181 183 185 186 188 189 19 190 195 196 197 198 199
 201 202 203 204 205 206 207 208 209 210 211 212 213 24 25 28 30 32 36 37
 38 4 41 42 43 45 47 48 49 5 54 56 58 59 6 61 63 66 67 7 70 72 74 75 80 81
 83 85 88 9 90 91 92 93 94 96 97 98 99]
Index(['run_id', 'condition', 'rt', 'stimulus', 'key_press', 'trial_type',
       'trial_index', 'time_elapsed', 'internal_node_id', 'subject',
       'chinFirst', 'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'et_data', 'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
    

In [55]:
data_raw['degree'] = data_raw['degree'].replace(
    ['-3.0', '0', '1.25', '1.5', '2', '4.25'], np.nan)


data_raw['degree'] = data_raw['degree'].replace({
    'Middle School': 'middle',
    'High School': 'high',
    'high school': 'high',
    'Associate ': 'high',
    'masters degree': 'grad',
    'College / Undergraduate / Bachelor': 'college',
    'Bachelor': 'college',
    'Grad': 'grad',
    'Graduate / PhD / Master': 'grad',
    'masters degree': 'grad'
})
data_raw['degree'].unique()

array(['high', 'college', nan, 'grad', 'Associates Degree', 'highSchool',
       'graduate', 'middleSchool', 'middle'], dtype=object)

In [56]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)

columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_raw[columns] = data_raw[columns].replace({'no': 0, 'yes': 1}) 

data_raw['run_id'].unique()

array([1, 103, 106, 108, 11, 12, 124, 125, 126, 128, 13, 130, 131, 14,
       141, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157,
       160, 161, 162, 163, 164, 165, 166, 168, 171, 172, 173, 176, 177,
       178, 179, 180, 181, 183, 185, 186, 188, 189, 19, 190, 195, 196,
       197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
       211, 212, 213, 24, 25, 28, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45,
       47, 48, 49, 5, 54, 56, 58, 59, 6, 61, 63, 66, 67, 7, 70, 72, 74,
       75, 80, 81, 83, 85, 88, 9, 90, 91, 92, 93, 94, 96, 97, 98, 99],
      dtype=object)

# data_trial

In [65]:
data_trial = data_raw.loc[
    :,
    [
        'run_id', 'subject', 'prolificID', 'chinFirst', 
        'trial_index',
        'trial_type', 'task_nr', 
        'rt', 'stimulus', 'key_press', 
        'time_elapsed', 'trial_duration', 'recorded_at',
        'window_width', 'window_height', 'success', 

        'chin', 'x_pos', 'y_pos', 
        
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'chosenAmount', 'chosenDelay',
    ]
]

## Clean

### Failed Webgazer Setup

In [58]:
subjects_failedSetups = []
for subject in data_trial['run_id'].unique():
    maxTrialIndex = data_trial.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

data_trial.loc[data_trial['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID'
    ]
].drop_duplicates()    

,run_id,prolificID
1036,106,5d485e8415055400194b707f
1038,108,55b237e6fdf99b19ea79d2f7
2079,124,56b7a271e77ebe000bbeff49
4684,14,5ec2cdfd1a17930ddf6f1443
5782,147,5d754a0ae8351c0001f3ffc8
5786,148,5d754a0ae8351c0001f3ffc8
5790,149,5de80e0b521f95791f912e49
8488,156,5ff7baf4cbc6b34ca5779e90
9564,161,5f493291e3e64832df458323
12251,168,5c95970cd676900016e1a940


In [59]:
data_trial = data_trial.loc[~data_trial['run_id'].isin(subjects_failedSetups), :]
print(data_trial['run_id'].unique())

[1 103 11 12 125 126 128 13 130 131 141 144 146 150 151 152 153 154 157
 160 162 163 164 165 166 171 176 177 179 180 183 186 188 189 19 190 195
 196 197 198 199 201 203 205 206 207 208 209 210 211 212 213 24 25 28 30
 32 36 37 38 4 41 42 43 45 47 48 5 56 58 59 6 61 63 66 67 7 70 74 75 80 81
 85 9 90 91 92 93 94 96 97 98]


### Duplicate Prolific ID's

In [60]:
data_trial.loc[data_trial['run_id']==2002, 'prolificID'] = 'Tim2'

duplicates = data_trial.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_trial.loc[duplicates, ['run_id', 'prolificID', 'trial_index']]

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

Success: No duplicate subjects found


### Numerics

In [70]:
data_trial = convertToNumeric(data_trial, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height'
    ]
)

# data_et

## Compile

In [17]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

#for subject in data_yang['run_id'].unique():
#    for i in data_raw.loc[
#        (data_raw['run_id']==subject) & 
#        (pd.notna(data_raw['et_data'])) &
#        ~(data_raw['et_data'].isin(['"', 'nan'])), 
#       :].index:
#       print('Reformat index: ' + str(i))
#       data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

In [18]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)

def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[
        (pd.notna(data['et_data'])) & 
        ~(data['et_data'].isin(['"', '[]', 'nan'])), 
        :].index:
        
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw.loc[data_raw['run_id'].isin(data_trial['run_id'].unique()), :])

<ipython-input-18-80ee6abdb188>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["et_data"] = data['et_data'].apply(str)


Start decoding Index 17
Index 17 extracted
Start decoding Index 19
Index 19 extracted
Start decoding Index 21
Index 21 extracted
Start decoding Index 23
Index 23 extracted
Start decoding Index 25
Index 25 extracted
Start decoding Index 27
Index 27 extracted
Start decoding Index 29
Index 29 extracted
Start decoding Index 31
Index 31 extracted
Start decoding Index 33
Index 33 extracted
Start decoding Index 35
Index 35 extracted
Start decoding Index 37
Index 37 extracted
Start decoding Index 39
Index 39 extracted
Start decoding Index 41
Index 41 extracted
Start decoding Index 43
Index 43 extracted
Start decoding Index 45
Index 45 extracted
Start decoding Index 47
Index 47 extracted
Start decoding Index 49
Index 49 extracted
Start decoding Index 51
Index 51 extracted
Start decoding Index 53
Index 53 extracted
Start decoding Index 55
Index 55 extracted
Start decoding Index 57
Index 57 extracted
Start decoding Index 59
Index 59 extracted
Start decoding Index 61
Index 61 extracted
Start decod

Index 357 extracted
Start decoding Index 358
Index 358 extracted
Start decoding Index 360
Index 360 extracted
Start decoding Index 361
Index 361 extracted
Start decoding Index 363
Index 363 extracted
Start decoding Index 364
Index 364 extracted
Start decoding Index 366
Index 366 extracted
Start decoding Index 367
Index 367 extracted
Start decoding Index 369
Index 369 extracted
Start decoding Index 370
Index 370 extracted
Start decoding Index 372
Index 372 extracted
Start decoding Index 373
Index 373 extracted
Start decoding Index 375
Index 375 extracted
Start decoding Index 376
Index 376 extracted
Start decoding Index 378
Index 378 extracted
Start decoding Index 379
Index 379 extracted
Start decoding Index 381
Index 381 extracted
Start decoding Index 382
Index 382 extracted
Start decoding Index 393
Index 393 extracted
Start decoding Index 395
Index 395 extracted
Start decoding Index 397
Index 397 extracted
Start decoding Index 399
Index 399 extracted
Start decoding Index 401
Index 401 

Index 793 extracted
Start decoding Index 795
Index 795 extracted
Start decoding Index 796
Index 796 extracted
Start decoding Index 798
Index 798 extracted
Start decoding Index 799
Index 799 extracted
Start decoding Index 801
Index 801 extracted
Start decoding Index 802
Index 802 extracted
Start decoding Index 804
Index 804 extracted
Start decoding Index 805
Index 805 extracted
Start decoding Index 807
Index 807 extracted
Start decoding Index 808
Index 808 extracted
Start decoding Index 810
Index 810 extracted
Start decoding Index 811
Index 811 extracted
Start decoding Index 813
Index 813 extracted
Start decoding Index 814
Index 814 extracted
Start decoding Index 816
Index 816 extracted
Start decoding Index 817
Index 817 extracted
Start decoding Index 819
Index 819 extracted
Start decoding Index 820
Index 820 extracted
Start decoding Index 822
Index 822 extracted
Start decoding Index 823
Index 823 extracted
Start decoding Index 825
Index 825 extracted
Start decoding Index 826
Index 826 

Index 1152 extracted
Start decoding Index 1153
Index 1153 extracted
Start decoding Index 1155
Index 1155 extracted
Start decoding Index 1156
Index 1156 extracted
Start decoding Index 1158
Index 1158 extracted
Start decoding Index 1159
Index 1159 extracted
Start decoding Index 1161
Index 1161 extracted
Start decoding Index 1162
Index 1162 extracted
Start decoding Index 1164
Index 1164 extracted
Start decoding Index 1165
Index 1165 extracted
Start decoding Index 1167
Index 1167 extracted
Start decoding Index 1168
Index 1168 extracted
Start decoding Index 1170
Index 1170 extracted
Start decoding Index 1171
Index 1171 extracted
Start decoding Index 1184
Index 1184 extracted
Start decoding Index 1186
Index 1186 extracted
Start decoding Index 1188
Index 1188 extracted
Start decoding Index 1190
Index 1190 extracted
Start decoding Index 1192
Index 1192 extracted
Start decoding Index 1194
Index 1194 extracted
Start decoding Index 1196
Index 1196 extracted
Start decoding Index 1198
Index 1198 ex

...

Index 45072 extracted
Start decoding Index 45074
Index 45074 extracted
Start decoding Index 45076
Index 45076 extracted
Start decoding Index 45078
Index 45078 extracted
Start decoding Index 45080
Index 45080 extracted
Start decoding Index 45082
Index 45082 extracted
Start decoding Index 45084
Index 45084 extracted
Start decoding Index 45086
Index 45086 extracted
Start decoding Index 45097
Index 45097 extracted
Start decoding Index 45098
Index 45098 extracted
Start decoding Index 45100
Index 45100 extracted
Start decoding Index 45101
Index 45101 extracted
Start decoding Index 45103
Index 45103 extracted
Start decoding Index 45104
Index 45104 extracted
Start decoding Index 45106
Index 45106 extracted
Start decoding Index 45107
Index 45107 extracted
Start decoding Index 45109
Index 45109 extracted
Start decoding Index 45110
Index 45110 extracted
Start decoding Index 45112
Index 45112 extracted
Start decoding Index 45113
Index 45113 extracted
Start decoding Index 45115
Index 45115 extracte

Index 45397 extracted
Start decoding Index 45399
Index 45399 extracted
Start decoding Index 45401
Index 45401 extracted
Start decoding Index 45403
Index 45403 extracted
Start decoding Index 45405
Index 45405 extracted
Start decoding Index 45407
Index 45407 extracted
Start decoding Index 45409
Index 45409 extracted
Start decoding Index 45411
Index 45411 extracted
Start decoding Index 45413
Index 45413 extracted
Start decoding Index 45415
Index 45415 extracted
Start decoding Index 45417
Index 45417 extracted
Start decoding Index 45419
Index 45419 extracted
Start decoding Index 45421
Index 45421 extracted
Start decoding Index 45423
Index 45423 extracted
Start decoding Index 45425
Index 45425 extracted
Start decoding Index 45427
Index 45427 extracted
Start decoding Index 45429
Index 45429 extracted
Start decoding Index 45431
Index 45431 extracted
Start decoding Index 45433
Index 45433 extracted
Start decoding Index 45435
Index 45435 extracted
Start decoding Index 45437
Index 45437 extracte

Index 45743 extracted
Start decoding Index 45745
Index 45745 extracted
Start decoding Index 45746
Index 45746 extracted
Start decoding Index 45748
Index 45748 extracted
Start decoding Index 45749
Index 45749 extracted
Start decoding Index 45751
Index 45751 extracted
Start decoding Index 45752
Index 45752 extracted
Start decoding Index 45754
Index 45754 extracted
Start decoding Index 45755
Index 45755 extracted
Start decoding Index 45757
Index 45757 extracted
Start decoding Index 45758
Index 45758 extracted
Start decoding Index 45760
Index 45760 extracted
Start decoding Index 45761
Index 45761 extracted
Start decoding Index 45763
Index 45763 extracted
Start decoding Index 45764
Index 45764 extracted
Start decoding Index 45766
Index 45766 extracted
Start decoding Index 45767
Index 45767 extracted
Start decoding Index 45769
Index 45769 extracted
Start decoding Index 45770
Index 45770 extracted
Start decoding Index 45772
Index 45772 extracted
Start decoding Index 45773
Index 45773 extracte

Index 46070 extracted
Start decoding Index 46072
Index 46072 extracted
Start decoding Index 46074
Index 46074 extracted
Start decoding Index 46076
Index 46076 extracted
Start decoding Index 46078
Index 46078 extracted
Start decoding Index 46080
Index 46080 extracted
Start decoding Index 46082
Index 46082 extracted
Start decoding Index 46084
Index 46084 extracted
Start decoding Index 46086
Index 46086 extracted
Start decoding Index 46088
Index 46088 extracted
Start decoding Index 46090
Index 46090 extracted
Start decoding Index 46092
Index 46092 extracted
Start decoding Index 46094
Index 46094 extracted
Start decoding Index 46096
Index 46096 extracted
Start decoding Index 46098
Index 46098 extracted
Start decoding Index 46100
Index 46100 extracted
Start decoding Index 46102
Index 46102 extracted
Start decoding Index 46104
Index 46104 extracted
Start decoding Index 46106
Index 46106 extracted
Start decoding Index 46108
Index 46108 extracted
Start decoding Index 46110
Index 46110 extracte

Index 46399 extracted
Start decoding Index 46401
Index 46401 extracted
Start decoding Index 46402
Index 46402 extracted
Start decoding Index 46404
Index 46404 extracted
Start decoding Index 46405
Index 46405 extracted
Start decoding Index 46407
Index 46407 extracted
Start decoding Index 46408
Index 46408 extracted
Start decoding Index 46410
Index 46410 extracted
Start decoding Index 46411
Index 46411 extracted
Start decoding Index 46413
Index 46413 extracted
Start decoding Index 46414
Index 46414 extracted
Start decoding Index 46416
Index 46416 extracted
Start decoding Index 46417
Index 46417 extracted
Start decoding Index 46420
Index 46420 extracted
Start decoding Index 46421
Index 46421 extracted
Start decoding Index 46423
Index 46423 extracted
Start decoding Index 46424
Index 46424 extracted
Start decoding Index 46426
Index 46426 extracted
Start decoding Index 46427
Index 46427 extracted
Start decoding Index 46429
Index 46429 extracted
Start decoding Index 46430
Index 46430 extracte

In [19]:
data_et = convertToNumeric(data_et, ['x', 'y', 't', 't_task'])

In [20]:
data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()

<ipython-input-20-5a412f0f98d6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()


x   y
run_id trial_index        
1.0    18.0          6   6
       20.0         11  11
       22.0         11  11
       24.0         11  11
       26.0         10  10
...                 ..  ..
213.0  520.0        40  40
       522.0        21  21
       523.0        41  41
       525.0        21  21
       526.0        41  41

[22992 rows x 2 columns]

# data_subject

In [21]:
df1 = data_raw.loc[
    :, 
    [
        'run_id',
        'age', 'browliner', 'browser', 'browser_version', 'degree', 'device',
        'ethnic', 'eyeliner', 'eyeshadow', 'gender', 'glasses', 'keptHead',
        'masquara', 'optionalNote', 'platform', 'platform_version', 'sight',
        'triedChin', 'user_agent', 'vertPosition', 'webcam_fps',
        'webcam_height', 'webcam_label', 'webcam_width', 'chosenAmount',
        'chosenDelay'
    ]
].drop_duplicates()
    
df2 = data_trial.loc[
    : ,
    [
        'chinFirst', 'choiceTask_amountLeftFirst', 'run_id', 'prolificID'
    ]
].drop_duplicates()
    
data_subject = df1.merge(df2, on='run_id', how='left') \
    .rename(columns={'age': 'birthyear'})

## Add prolific data

In [22]:
data_prolific = pd.read_csv(
        r'C:/Users/User/GitHub/WebET_Analysis/prolific/prolific_export.csv'
    ).rename(columns={'participant_id': 'prolificID'})
print(len(
    data_prolific.loc[data_prolific['status']=='APPROVED', :]))
print(len(data_subject.prolificID.unique()))




21
93


In [23]:
data_subject = data_subject.merge(data_prolific, on='prolificID', how='left')

data_subject['status'] = data_subject['status'].fillna('NOTPROLIFIC')

data_subject.loc[:, ['run_id', 'prolificID']]

,run_id,prolificID
0,1,5fccc8ac636416a4288a9f3d
1,103,600063f2943eab0acc812ed8
2,106,NaN
3,108,NaN
4,11,5c5684ef9d244c0001b29f1e
...,...,...
110,94,5ecfc227f036c902457fc44c
111,96,5eb9940830ab6c098bd1943d
112,97,5f68fe5be5cc370c0a0911de
113,98,5f779405dacc842fc44bc767


# data_prolific
For participant management

In [24]:
temp = data_subject.rename(columns={
    'chosenAmount': 'bonus_USD',
    'chosenDelay': 'bonus_delay'
    }
)

data_prolific = data_prolific.merge(
    temp.loc[:, 
        np.append(['prolificID'], 
        temp.columns.difference(data_prolific.columns))], 
    on='prolificID', 
    how='left')

# Export data

In [25]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)
data_prolific.to_csv("data_jupyter/data_prolific.csv", index=False, header=True)

# Feedback

In [26]:
print('Success! Script ran through')

Success! Script ran through
